Projet AirBnB
==============================
Membres du groupe :
- Célia BEDEL (TD M)
- Maé BENAROCHE (TD D)

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
# Charger les données
train_data = pd.read_csv("data/airbnb_train.csv")
airbnb= pd.read_csv("data/airbnb_train.csv")
test_data = pd.read_csv("data/airbnb_test.csv")

# 0. Introduction

In [ ]:
train_data.head()

In [ ]:
#Visualisons les données
train_data.info()
#train_data.describe()

In [ ]:
airbnb.isnull().sum()

Cela nous donne une première idée sur notre jeu de données : 28 colonnes et  22 234 entrées. Avant toute chose, on peut déjà éliminer plusieurs colonnes qui n'ont, à priori, pas d'impact sur le prix : id, zipcode, name (unique à chaque logement).
Pour pouvoir utiliser toutes les données restantes, il faut les convertir en données numériques. Ainsi, on pourra plus facilement afficher des graphes et effectuer des moyennes, des matrices de corrélations, etc. C'est l'objectif des classes Custom Transformation et Feature Selection.

# 1. Exploration des données

## CustomTransformation

In [6]:
class CustomTransformation():

    def __init__(self):
        """
        Classe pour convertir les colonnes en valeurs qui vont pouvoir être converties en valeurs numériques
        """

        self.fitted = False # Indique si fit_transform a été utilisé, pour éviter d’utiliser transform sans que fit ai été appelé
        self.property2index = dict() # Dictionnaire qui va convertir le nom en indice
        self.max_index = 0 # Indique le dernier indice de la propriété.

    def fit_transform(self, dataset):

        self.fitted = True
        # transform
        return self.transform(dataset)

    def transform(self, dataset):

        # Tranforme les types de propriétés en indice
        dataset = self.transform_property(dataset, "property_type")
        dataset = self.transform_property(dataset, "room_type")
        dataset = self.transform_property(dataset, "bed_type")
        dataset = self.transform_property(dataset, "cancellation_policy")

        # Transforme les colonnes qui prennent que deux valeurs : "host_has_profile_pic", "host_identity_verifie", "instant_bookable"
        dataset = self.transform_bool(dataset)
        dataset=self.transform_host_response_rate(dataset)

        #transforme les colonnes qui ont pour valeurs des dates
        dataset=self.transform_date(dataset)

        dataset = self.transform_amenities(dataset)

        dataset = self.transform_amen(dataset)

        # remplace les valeurs null
        dataset['accommodates'].fillna(0, inplace=True)
        dataset['bathrooms'].fillna(0, inplace=True)
        dataset['latitude'].fillna(0, inplace=True)
        dataset['longitude'].fillna(0, inplace=True)
        dataset['number_of_reviews'].fillna(0, inplace=True)
        dataset['review_scores_rating'].fillna(0, inplace=True)
        dataset['bedrooms'].fillna(0, inplace=True)
        dataset['beds'].fillna(0, inplace=True)
        dataset['host_response_rate'].fillna(0, inplace=True)

        dataset.fillna(0, inplace=True)

        return dataset

    #Class pour convertir les colonnes du type : 'type de quelque chose' en des indices numériques
    def transform_property(self, dataset, property_name):
        # Récupère les types de propriété (maison, appart etc...)
        properties = dataset[property_name].unique()
        print("Tous les types de ", property_name, properties)

        self.property2index = {prop: i for (i, prop) in enumerate(properties)}
        self.max_index = max(list(self.property2index.values()))

        # Transform les propriétés en indice
        dataset.loc[:, property_name] = dataset[property_name].replace(self.property2index)

        # Ligne un peu moche qui fait en sorte de remplacer les lignes qui ont des nom de logement qui n’était pas dans l’entrainement
        dataset.loc[dataset[property_name].map(type).eq(str), property_name] = np.nan

        # remplace les valeurs null
        dataset[dataset[property_name].isna()] = self.max_index + 1

        return dataset

    #Transforme les colonnes qui ne prennent que deux entrées : true/false, t/f
    def transform_bool(self,dataset):
        #Si la valeur est t, la valeur devient 1, sinon (y compris valeur nulle) la valeur devient 0
        dataset.loc[:, "cleaning_fee"] = dataset["cleaning_fee"].map(lambda x: 1 if x == 't' else 0)
        dataset.loc[:, "host_has_profile_pic"] = dataset["host_has_profile_pic"].map(lambda x: 1 if x == 't' else 0)
        dataset.loc[:, "host_identity_verified"] = dataset["host_identity_verified"].map(lambda x: 1 if x == 't' else 0)
        dataset.loc[:, "instant_bookable"] = dataset["instant_bookable"].map(lambda x: 1 if x == 't' else 0)
        return dataset

    #Transforme la colonne host_response_rate
    def transform_host_response_rate(self, dataset):
      #Enlève le symbole '%' et converti en float
      dataset.loc[:, 'host_response_rate'] = dataset['host_response_rate'].str.rstrip('%').astype(float)
      # Diviser par 100 pour obtenir la valeur sous forme de pourcentage
      dataset['host_response_rate'] = dataset['host_response_rate'] / 100

      return dataset

    #Converti les colonnes avec des dates en nombre de jours par rapport à la date d'aujourd'hui
    def transform_date(self, dataset):

      # Convertir la colonne 'host_since' en objets de date
      dataset['host_since'] = pd.to_datetime(dataset['host_since'])
      # Calculer la différence en jours entre chaque date dans la colonne 'host_since' et aujourd'hui
      dataset['host_since'] = (datetime.now() - dataset['host_since']).dt.days

      dataset['first_review'] = pd.to_datetime(dataset['first_review'])
      dataset['first_review'] = (datetime.now() - dataset['first_review']).dt.days

      dataset['last_review'] = pd.to_datetime(dataset['last_review'])
      dataset['last_review'] = (datetime.now() - dataset['last_review']).dt.days

      return dataset

    def transform_amenities(self, dataset):
        amenities_dictonary = {}
        for i in range(len(dataset)):
            amenities = dataset["amenities"][i]
            amenities = (amenities.replace("{", "").replace("}", "").replace('"', "").split(","))
            for amenity in amenities:
                if amenity not in amenities_dictonary:
                    amenities_dictonary[amenity] = len(amenities_dictonary)

        print("Tous les types d'amenities :" ,amenities_dictonary)

        return dataset


    #on regroupe dans des catégories les amenities (bathroom, kitchen, essentials, etc.)
    def transform_amen(self, dataset):
        categories = {
        'bathroom': ['Shampoo', 'Hair dryer','Washer','Dryer','Iron','Hot water'],
        'bedroom': ['Extra pillows and blankets', 'Bed linens'],
        'kitchen': ['Kitchen', 'Refrigerator', 'Stove', 'Oven', 'Microwave', 'Dishwasher', 'Dishes and silverware', 'Cooking basics', 'Coffee maker'],
        'heating': ['Heating', 'Air conditioning'],
        'outdoor': ['Patio or balcony', 'Garden or backyard', 'BBQ grill', 'Outdoor furniture'],
        'tv/wifi': ['TV', 'Cable TV', 'Wireless Internet'],
        'smoking': ['Smoking allowed', 'Smoke detector', 'Carbon monoxide detector','Fire extinguisher'],
        'security':['Lock on bedroom door','Private entrance','24-hour check-in'],
        'pets':['Pets allowed','Pets live on this property','Dog(s)'],
        # a rajouter ou non 'parking':['Free parking on premises'],
        'essentials': ['Essentials', 'Bed linens', 'Extra pillows and blankets', 'Hangers', 'Hair dryer', 'Iron', 'Shampoo']
        }

        # Initialiser les colonnes pour chaque catégorie
        for category in categories:
            dataset[category] = 0

        def count_amenities_by_category(amenities, categories):
            if not isinstance(amenities, str):
                return {category: 0 for category in categories.keys()}

            amenity_list = amenities.strip('{}').split(',')
            amenity_list = [amenity.strip().strip('"') for amenity in amenity_list]
            category_counts = {category: 0 for category in categories.keys()}

            for amenity in amenity_list:
                for category, keywords in categories.items():
                    if any(keyword in amenity for keyword in keywords):
                        category_counts[category] += 1  # <-- Modification ici pour compter le nombre d'amenities par catégorie

            return category_counts

        # Appliquer la fonction à chaque ligne et mettre à jour les colonnes du dataset
        for index, row in dataset.iterrows():
            category_counts = count_amenities_by_category(row['amenities'], categories)
            for category, count in category_counts.items():
                dataset.at[index, category] = count

        return dataset




In [ ]:
features_transformer = CustomTransformation()

airbnb_train_transformed = features_transformer.fit_transform(airbnb)
airbnb_train_transformed.head()

In [ ]:
airbnb_train_transformed.info()

## FeatureSelection

In [9]:
class FeatureSelection():

    def __init__(self, columns):
        """
        Class simple pour juste garder les colonnes qui nous intéresse
        N'a pas forcément l'air nécessaire, mais c'est pour être sur que j'applique bien le même process au train et au test
        """

        self.columns = columns

    def fit_transform(self, dataset, y=None):
        return self.transform(dataset)

    def transform(self, dataset):
        if not self.columns:
            raise ValueError("Aucune colonne spécifiée pour la sélection de caractéristiques.")

        new_dataset = dataset[self.columns].copy()

        for col in new_dataset.columns:
            new_dataset[col] = pd.to_numeric(new_dataset[col]) # Converti tout en nombre
        return new_dataset


In [ ]:
columns1 = ["property_type","room_type", "bathrooms", "bed_type", "cancellation_policy", "cleaning_fee","first_review","host_has_profile_pic",
           "host_identity_verified","host_response_rate","host_since","instant_bookable", "last_review", "latitude","longitude","number_of_reviews","review_scores_rating",
            "bedrooms","beds", "bathroom","bedroom","kitchen","heating","outdoor","tv/wifi","smoking","security","pets","essentials","log_price"]





feature_selector = FeatureSelection(columns1)

airbnb_train = feature_selector.transform(airbnb_train_transformed)
airbnb_train.head()

Maintenant que notre jeu de données est sous forme de valeurs numériques, on va pouvoir étudier les données. C'est à dire regarder quelles variables (colonnes) sont utiles pour la prédiction du prix ( et donc du log du prix).

## Affichage des données sous forme de graphiques, matrice de corrélation, etc.

### Matrice de corrélation

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

airbnb_train_matrix = airbnb_train.copy()
airbnb_train_matrix["log_price"] = train_data["log_price"]
airbnb_train_matrix['price'] = np.exp(airbnb_train_matrix['log_price'])

# Calculer la matrice de corrélation
correlation_matrix = airbnb_train_matrix.corr()

# Afficher la matrice de corrélation
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Matrice de corrélation")
plt.show()

In [ ]:
#pour afficher seulement la correlation avec log_price
airbnb_train_matrix = airbnb_train.copy()
airbnb_train_matrix["log_price"] = train_data["log_price"]

# Calculer la matrice de corrélation
correlation_with_log_price = airbnb_train_matrix.corr()["log_price"].drop("log_price")
correlation_matrix = pd.DataFrame(correlation_with_log_price)


# Afficher la matrice de corrélation
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Matrice de corrélation")
plt.show()

On remarque sur la matrice de corrélation que les variables les plus corrélées avec le prix sont : bedrooms, beds, room_type, bathrooms et tv/wifi.
On peut donc en conclure que les clients sont prêts à payer plus cher pour un logement avec plus de chambres, de lits, de salles de bain, une meilleure qualité de wifi et de tv.

### Carte

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

# Calculer le prix à partir du log_price
airbnb_train['price'] = np.exp(airbnb_train['log_price'])

# Créer un GeoDataFrame à partir des données
gdf = gpd.GeoDataFrame(airbnb_train, geometry=gpd.points_from_xy(airbnb_train.longitude, airbnb_train.latitude))

# Charger un fond de carte du monde
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Tracer la carte
fig, ax = plt.subplots(figsize=(15, 10))
world.plot(ax=ax, color='lightgray')

# Ajouter les points
gdf.plot(ax=ax, marker='o', c=gdf['log_price'], cmap='Reds', markersize=10, alpha=0.6)

# Ajouter une barre de couleur
sm = plt.cm.ScalarMappable(cmap='Reds', norm=plt.Normalize(vmin=gdf['log_price'].min(), vmax=gdf['log_price'].max()))
sm._A = []
cbar = plt.colorbar(sm, ax=ax)
cbar.set_label(r'$\log_{10}({\rm price})$')

# Afficher la carte
plt.title('Carte des prix en fonction de la latitude et de la longitude')
plt.show()

On remarque sur la carte que les airbnb sont très concentrés au même pays (états unis) donc qu'il n'y a pas forcément un gros impact de la localisation sur le prix.

### Graphiques

In [14]:
import seaborn as sns
import matplotlib.pyplot as plt

# Fonction qui permet de créer un graphique de la colonne spécifiée par rapport au prix
def plot_column_vs_price(dataset, column):
    plt.figure(figsize=(10, 6))
    if dataset[column].dtype == 'object':
        # Pour les colonnes catégorielles, on utilise un box plot
        sns.boxplot(x=column, y='log_price', data=dataset)
        plt.xticks(rotation=90)
    else:
        # Pour les colonnes numériques, on utilise un scatter plot
        sns.scatterplot(x=column, y='log_price', data=dataset)

    plt.title(f'Prix en fonction de {column}')
    plt.xlabel(column)
    plt.ylabel('logarithme du prix')
    plt.show()

In [ ]:
# Visualiser tous les graphiques
for column in airbnb_train.columns:
    if column != 'log_price':
        plot_column_vs_price(airbnb_train, column)

On a obtenu plusieurs graphiques qui nous permettent de voir la répartition des prix en fonction de certaines variables.

On peut voir que les prix sont plus élevés pour les logements entiers que pour les chambres privées ou partagées. On peut aussi voir que les prix sont plus élevés pour les logements avec plus de chambres, de lits, de salles de bain, etc.

Plus le review_scores_rating est élevé, plus le prix est élevé car les clients ont tendance à beacoup se fier aux notes afin de choisir un airbnb.
On remarque également que plus le nombre de reviews est élevé, plus le prix est élevé. Cela peut s'expliquer par le fait que les logements avec beaucoup de reviews sont plus populaires et donc plus chers.

Les clients recherchent un logement qui est sûr et dont le propriétaire a l'air fiable. C'est pourquoi les prix sont plus élevés pour les logements avec des superhosts qui ont une photo de profil et des verified hosts. Ils regardent également la catégorie host_since qui est un indicateur de la fiabilité du propriétaire. La catégorie host_response_time est également un indicateur de la fiabilité du propriétaire. Les clients recherchent un propriétaire qui répond rapidement à leurs messages.

On remarque également que les prix sont plus élevés pour les logements avec des équipements de qualité comme une cuisine, un lave-linge, un sèche-linge, etc. Plus le logement est équipé, plus le prix est élevé.



## Différents jeux de données

On va extraire différents dataset de notre dataset princpal pour après tester plusieurs modèles sur chacun d'entre eux.

### Dataset avec toutes les données

In [ ]:
columns1 = ["property_type","room_type", "bathrooms", "bed_type", "cancellation_policy", "cleaning_fee","first_review","host_has_profile_pic",
           "host_identity_verified","host_response_rate","host_since","instant_bookable", "last_review", "latitude","longitude","number_of_reviews","review_scores_rating",
            "bedrooms","beds", "bathroom","bedroom","kitchen","heating","outdoor","tv/wifi","smoking","security","pets","essentials","log_price"]





feature_selector = FeatureSelection(columns1)

airbnb_train = feature_selector.transform(airbnb_train_transformed)
airbnb_train.head()

### Dataset filtré : toutes les colonnes mais en enlevant les lignes qui ont trop de 0

In [ ]:
#on enlève les lignes qui ont trop de 0

# Spécifier le seuil de valeurs nulles autorisé par ligne
seuil_zeros = 5
# Assurez-vous que les valeurs non numériques sont converties en NaN pour éviter les erreurs
#airbnb = airbnb.apply(pd.to_numeric, errors='coerce')

# Compter le nombre de zéros par ligne
nb_zeros_par_ligne = (airbnb_train == 0).sum(axis=1)

# Afficher quelques lignes avec plus de seuil_zeros (pour vérification)
#print(airbnb[nb_zeros_par_ligne > seuil_zeros].head())

# Filtrer les lignes en fonction du nombre de zéros
airbnb_filtre = airbnb_train[nb_zeros_par_ligne <= seuil_zeros]

# Afficher les résultats
print("Nombre d'entrées avant :", len(airbnb_train))
print("Nombre d'entrées après :", len(airbnb_filtre))
print("Pourcentage d'entrées restantes :", (len(airbnb_filtre) / len(airbnb_train)) * 100, "%")


### Dataset en fonction de matrice de corrélation

Pour ce dataset, le but est de garder les variables qui ont une forte corrélation avec log_price. Nous avons ces informations grâce aux matrices de corrélations affichées plus haut. Nous allons prendre les colonnes qui ont un coefficient dans la matrice > 0,20.

In [ ]:
columns2 = ["room_type", "bathrooms", "bedrooms","beds", "tv/wifi","log_price"]

feature_selector = FeatureSelection(columns2)

airbnb_matrice_corr = feature_selector.transform(airbnb_train_transformed)
airbnb_matrice_corr.head()

### Dataset en fonction des graphiques

D'après les représentations graphiques, les variables qui nous paraissent importantes sont celles sur les type de logement, la taille du logement (nombre de salles de bain, de chambre, de lits), la fiabilité de l'hôte, les avis, les équipements majeurs (cuisine, salle de bain, internet).

In [ ]:
columns3 = ["property_type", "bathrooms","host_has_profile_pic",
           "host_identity_verified","host_since", "last_review", "number_of_reviews","review_scores_rating",
            "bedrooms","beds","kitchen","tv/wifi","log_price"]




feature_selector = FeatureSelection(columns3)

airbnb_graphique = feature_selector.transform(airbnb_train_transformed)
airbnb_graphique.head()

### Dataset intuitif : les variables qui nous paraitrait utiles pour déterminer le prix d'un AirBnB

In [ ]:
columns4 = ["property_type","room_type","bathrooms", "cleaning_fee","review_scores_rating","bedrooms","beds", "kitchen","tv/wifi","log_price"]


feature_selector = FeatureSelection(columns4)

airbnb_utile = feature_selector.transform(airbnb_train_transformed)
airbnb_utile.head()

# 2. Prédictions (Modèles)

## PCA

In [21]:
#importer les librairies
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
#importer les librairies
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

#standardiser les données
scaler=StandardScaler()
airbnb_train_std=scaler.fit_transform(airbnb_train)

#Créer le modèle de la PCA
pca=PCA()

#appliquer la PCA et projeter les données
airbnb_train_pca=pca.fit_transform(airbnb_train_std)

#obtenir les composantes principales (= les vect pp)
#print('Composantes principales :')
#print(pca.components_)

#obtenir les variances expliquées (=les val pp)
print('Variances expliquées :')
print(pca.explained_variance_)

#obtenir les proportions des variances expliquées (=les taux d'inertie)
print('Proportions des variances expliquées : ')
proportion_variance_expliquee=pca.explained_variance_ratio_
print(proportion_variance_expliquee)

In [ ]:
import matplotlib.pyplot as plt

# Calculer les proportions cumulatives de variance expliquée
cumulative_variance_ratio = proportion_variance_expliquee.cumsum()

# Tracer le graphique
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(cumulative_variance_ratio) + 1), cumulative_variance_ratio, marker='o', linestyle='-')
plt.title('Proportion cumulative de variance expliquée')
plt.xlabel('Nombre de composantes principales')
plt.ylabel('Proportion cumulative de variance expliquée')
plt.grid(True)
plt.show()

In [ ]:
# Créer une instance PCA avec 5 composantes principales
pca = PCA(n_components=5)

# Appliquer la PCA aux données standardisées
airbnb_train_pca = pca.fit_transform(airbnb_train_std)

# Créer un DataFrame à partir des composantes principales
airbnb_train_pca_df = pd.DataFrame(data=airbnb_train_pca, columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])
#airbnb_train_pca_df = pd.DataFrame(data=airbnb_train_pca, columns=['PC1', 'PC2'])

# Afficher les premières lignes du DataFrame des composantes principales
print(airbnb_train_pca_df.head())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

# Séparation des features et de la cible
X = airbnb_train.copy()
y = train_data['log_price']

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Standardisation des données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Application de la PCA
pca = PCA(n_components=5)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Entraînement du modèle de régression linéaire
model = LinearRegression()
model.fit(X_train_pca, y_train)

# Prédiction du log_price sur l'ensemble de test
y_pred_train=model.predict(X_train_pca)
y_pred_test = model.predict(X_test_pca)

# Calcul de l'erreur quadratique moyenne
mse_test = mean_squared_error(y_test, y_pred_test)
print("Erreur quadratique moyenne :", mse_test)

mse_train = mean_squared_error(y_train, y_pred_train)
print("Erreur quadratique moyenne :", mse_train)

from sklearn.metrics import r2_score

# Calculer le score R^2 sur l'ensemble d'entraînement
train_r2_score = r2_score(y_train, y_pred_train)

# Calculer le score R^2 sur l'ensemble de test
test_r2_score = r2_score(y_test, y_pred_test)

print("Score R^2 sur l'ensemble d'entraînement :", train_r2_score)
print("Score R^2 sur l'ensemble de test :", test_r2_score)


print(f"Score en entrainenement : {r2_score(y_true=y_train, y_pred=y_pred_train)}")
print(f"Score en test : {r2_score(y_true=y_test, y_pred=y_pred_test)}")

## Modèle Regression linéaire multiple

### Régression linéaire avec tout le modèle

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

# Séparation des features et de la cible
X = airbnb_train.copy()
y = train_data['log_price']

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Entraînement du modèle de régression linéaire
model = LinearRegression()
model.fit(X_train, y_train)

# Prédiction du log_price sur l'ensemble de test
y_pred_train=model.predict(X_train)
y_pred_test = model.predict(X_test)

# Calcul de l'erreur quadratique moyenne
mse_test = mean_squared_error(y_test, y_pred_test)
print("Erreur quadratique moyenne en test :", mse_test)

mse_train = mean_squared_error(y_train, y_pred_train)
print("Erreur quadratique moyenne en train :", mse_train)

from sklearn.metrics import r2_score

print(f"Score en entrainenement : {r2_score(y_true=y_train, y_pred=y_pred_train)}")
print(f"Score en test : {r2_score(y_true=y_test, y_pred=y_pred_test)}")

### Régression linéaire en enlevant les lignes qui ont trop de valeures nulles ou égale à 0, mais cela réduit de moitié la taille du dataset


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

# Séparation des features et de la cible
X = airbnb_filtre.copy()
X = X.drop(columns=['log_price'])
y = airbnb_filtre['log_price']

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Entraînement du modèle de régression linéaire
model = LinearRegression()
model.fit(X_train, y_train)

# Prédiction du log_price sur l'ensemble de test
y_pred_train=model.predict(X_train)
y_pred_test = model.predict(X_test)

# Calcul de l'erreur quadratique moyenne
mse_test = mean_squared_error(y_test, y_pred_test)
print("Erreur quadratique moyenne en test :", mse_test)

mse_train = mean_squared_error(y_train, y_pred_train)
print("Erreur quadratique moyenne en train :", mse_train)

from sklearn.metrics import r2_score

print(f"Score en entrainenement : {r2_score(y_true=y_train, y_pred=y_pred_train)}")
print(f"Score en test : {r2_score(y_true=y_test, y_pred=y_pred_test)}")

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(y_test, y_pred_test, alpha=0.3)
plt.xlabel("Valeurs réelles")
plt.ylabel("Valeurs prédites")
plt.title("Valeurs réelles vs Valeurs prédites")
plt.show()

### Regression lineaire avec toutes les entrées ; mais en séparant celles qui ont plus de 5 valeures nulles et les autres.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


# Spécifier le seuil de zéros autorisé par ligne
seuil_zeros = 3

# Assurez-vous que les valeurs non numériques sont converties en NaN pour éviter les erreurs
airbnb_train = airbnb_train.apply(pd.to_numeric, errors='coerce')

# Compter le nombre de zéros par ligne
nb_zeros_par_ligne = (airbnb_train == 0).sum(axis=1)

# Séparer les lignes en fonction du nombre de zéros
airbnb_train_filtre = airbnb_train[nb_zeros_par_ligne <= seuil_zeros]
airbnb_train_exclu = airbnb_train[nb_zeros_par_ligne > seuil_zeros]

# Séparation des features et de la cible pour les lignes filtrées
X_filtre = airbnb_train_filtre.drop(columns=['log_price'])
y_filtre = airbnb_train_filtre['log_price']

# Séparation des features et de la cible pour les lignes exclues
X_exclu = airbnb_train_exclu.drop(columns=['log_price'])
y_exclu = airbnb_train_exclu['log_price']

# Division des données en ensembles d'entraînement et de test
X_train_filtre, X_test_filtre, y_train_filtre, y_test_filtre = train_test_split(X_filtre, y_filtre, test_size=0.2, random_state=42)
X_train_exclu, X_test_exclu, y_train_exclu, y_test_exclu = train_test_split(X_exclu, y_exclu, test_size=0.2, random_state=42)

# Entraînement des modèles de régression linéaire
model_filtre = LinearRegression()
model_filtre.fit(X_train_filtre, y_train_filtre)

model_exclu = LinearRegression()
model_exclu.fit(X_train_exclu, y_train_exclu)

# Prédiction du log_price sur les ensembles de test
y_pred_test_filtre = model_filtre.predict(X_test_filtre)
y_pred_test_exclu = model_exclu.predict(X_test_exclu)

# Prédiction du log_price sur les ensembles d'entraînement
y_pred_train_filtre = model_filtre.predict(X_train_filtre)
y_pred_train_exclu = model_exclu.predict(X_train_exclu)

# Calcul de l'erreur quadratique moyenne pour les deux ensembles
mse_test_filtre = mean_squared_error(y_test_filtre, y_pred_test_filtre)
mse_test_exclu = mean_squared_error(y_test_exclu, y_pred_test_exclu)

mse_train_filtre = mean_squared_error(y_train_filtre, y_pred_train_filtre)
mse_train_exclu = mean_squared_error(y_train_exclu, y_pred_train_exclu)

print("Erreur quadratique moyenne en test (filtré) :", mse_test_filtre)
print("Erreur quadratique moyenne en test (exclu) :", mse_test_exclu)
print("Erreur quadratique moyenne en train (filtré) :", mse_train_filtre)
print("Erreur quadratique moyenne en train (exclu) :", mse_train_exclu)

# Combiner les prédictions
y_test_combined = np.concatenate((y_test_filtre, y_test_exclu))
y_pred_test_combined = np.concatenate((y_pred_test_filtre, y_pred_test_exclu))

y_train_combined = np.concatenate((y_train_filtre, y_train_exclu))
y_pred_train_combined = np.concatenate((y_pred_train_filtre, y_pred_train_exclu))

# Calcul des MSE pour les ensembles combinés
mse_combined_test = mean_squared_error(y_test_combined, y_pred_test_combined)
mse_combined_train = mean_squared_error(y_train_combined, y_pred_train_combined)

print("Erreur quadratique moyenne en test pour l'ensemble combiné :", mse_combined_test)
print("Erreur quadratique moyenne en train pour l'ensemble combiné :", mse_combined_train)


## Modèle Ridge

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge

# Séparation des features et de la cible
X = airbnb_train.copy()
X = X.drop(columns=['log_price'])
y = airbnb_train['log_price']

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Entraînement du modèle de régression linéaire
model = Ridge()
model.fit(X_train, y_train)

# Prédiction du log_price sur l'ensemble de test
y_pred_train=model.predict(X_train)
y_pred_test = model.predict(X_test)

# Calcul de l'erreur quadratique moyenne
mse_test = mean_squared_error(y_test, y_pred_test)
print("Erreur quadratique moyenne en test :", mse_test)

mse_train = mean_squared_error(y_train, y_pred_train)
print("Erreur quadratique moyenne en train :", mse_train)

from sklearn.metrics import r2_score

print(f"Score en entrainenement : {r2_score(y_true=y_train, y_pred=y_pred_train)}")
print(f"Score en test : {r2_score(y_true=y_test, y_pred=y_pred_test)}")


In [ ]:
#Avec GridSearch pour optimiser les paramètres

X_train, X_test, y_train, y_test = train_test_split(X, y)
model = Ridge()
model.fit(X_train,y_train)
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

from sklearn.model_selection import GridSearchCV
# Définir la grille de paramètres
parameters = {'alpha': [0.01, 0.1, 1, 10, 100]}
grid=GridSearchCV(model, parameters,cv=10)

# Exécuter la recherche
grid.fit(X_train, y_train)

print(grid.best_params_)

# Meilleurs paramètres et modèle
print(f"Best parameters: {grid.best_params_}")
best_ridge_model = grid.best_estimator_

print(f"jsp quoi : {best_ridge_model}")

# Réentraîner sur l'ensemble complet des données d'entraînement
best_ridge_model.fit(X_train, y_train)
y_pred_train = best_ridge_model.predict(X_train)
y_pred_test = best_ridge_model.predict(X_test)

# Calcul de l'erreur quadratique moyenne
mse_test = mean_squared_error(y_test, y_pred_test)
print("Erreur quadratique moyenne en test :", mse_test)
mse_train = mean_squared_error(y_train, y_pred_train)
print("Erreur quadratique moyenne en train :", mse_train)

#Calcule score r^2
print(f"Score en entrainenement : {r2_score(y_true=y_train, y_pred=y_pred_train)}")
print(f"Score en test : {r2_score(y_true=y_test, y_pred=y_pred_test)}")

## Modèle Lasso

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso

# Séparation des features et de la cible
X = airbnb_train.copy()
X = X.drop(columns=['log_price'])
y = airbnb_train['log_price']

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Entraînement du modèle de régression linéaire
model = Lasso()
model.fit(X_train, y_train)

# Prédiction du log_price sur l'ensemble de test
y_pred_train=model.predict(X_train)
y_pred_test = model.predict(X_test)

# Calcul de l'erreur quadratique moyenne
mse_test = mean_squared_error(y_test, y_pred_test)
print("Erreur quadratique moyenne en test :", mse_test)

mse_train = mean_squared_error(y_train, y_pred_train)
print("Erreur quadratique moyenne en train :", mse_train)

from sklearn.metrics import r2_score

print(f"Score en entrainenement : {r2_score(y_true=y_train, y_pred=y_pred_train)}")
print(f"Score en test : {r2_score(y_true=y_test, y_pred=y_pred_test)}")


## ElasticNet

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import ElasticNet

# Séparation des features et de la cible
X = airbnb_train.copy()
X = X.drop(columns=['log_price'])
y = airbnb_train['log_price']

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Entraînement du modèle de régression linéaire
model = ElasticNet()
model.fit(X_train, y_train)

# Prédiction du log_price sur l'ensemble de test
y_pred_train=model.predict(X_train)
y_pred_test = model.predict(X_test)

# Calcul de l'erreur quadratique moyenne
mse_test = mean_squared_error(y_test, y_pred_test)
print("Erreur quadratique moyenne en test :", mse_test)

mse_train = mean_squared_error(y_train, y_pred_train)
print("Erreur quadratique moyenne en train :", mse_train)

from sklearn.metrics import r2_score

print(f"Score en entrainenement : {r2_score(y_true=y_train, y_pred=y_pred_train)}")
print(f"Score en test : {r2_score(y_true=y_test, y_pred=y_pred_test)}")


## SVR

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR

# Séparation des features et de la cible
X = airbnb_train.copy()
X = X.drop(columns=['log_price'])
y = airbnb_train['log_price']

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Entraînement du modèle de régression linéaire
model = SVR()
model.fit(X_train, y_train)

# Prédiction du log_price sur l'ensemble de test
y_pred_train=model.predict(X_train)
y_pred_test = model.predict(X_test)

# Calcul de l'erreur quadratique moyenne
mse_test = mean_squared_error(y_test, y_pred_test)
print("Erreur quadratique moyenne en test :", mse_test)

mse_train = mean_squared_error(y_train, y_pred_train)
print("Erreur quadratique moyenne en train :", mse_train)

from sklearn.metrics import r2_score

print(f"Score en entrainenement : {r2_score(y_true=y_train, y_pred=y_pred_train)}")
print(f"Score en test : {r2_score(y_true=y_test, y_pred=y_pred_test)}")


## DecisionTreeRegressor

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor

# Séparation des features et de la cible
X = airbnb_train.copy()
X = X.drop(columns=['log_price'])
y = airbnb_train['log_price']

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Entraînement du modèle de régression linéaire
model = DecisionTreeRegressor()
model.fit(X_train, y_train)

# Prédiction du log_price sur l'ensemble de test
y_pred_train=model.predict(X_train)
y_pred_test = model.predict(X_test)

# Calcul de l'erreur quadratique moyenne
mse_test = mean_squared_error(y_test, y_pred_test)
print("Erreur quadratique moyenne en test :", mse_test)

mse_train = mean_squared_error(y_train, y_pred_train)
print("Erreur quadratique moyenne en train :", mse_train)

from sklearn.metrics import r2_score

print(f"Score en entrainenement : {r2_score(y_true=y_train, y_pred=y_pred_train)}")
print(f"Score en test : {r2_score(y_true=y_test, y_pred=y_pred_test)}")

## RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Séparation des features et de la cible
X = airbnb_train.copy()
X = X.drop(columns=['log_price'])
y = airbnb_train['log_price']

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Entraînement du modèle de régression linéaire
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Prédiction du log_price sur l'ensemble de test
y_pred_train=model.predict(X_train)
y_pred_test = model.predict(X_test)

# Calcul de l'erreur quadratique moyenne
mse_test = mean_squared_error(y_test, y_pred_test)
print("Erreur quadratique moyenne en test :", mse_test)

mse_train = mean_squared_error(y_train, y_pred_train)
print("Erreur quadratique moyenne en train :", mse_train)

from sklearn.metrics import r2_score

print(f"Score en entrainenement : {r2_score(y_true=y_train, y_pred=y_pred_train)}")
print(f"Score en test : {r2_score(y_true=y_test, y_pred=y_pred_test)}")

## GradientBoostingRegressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Séparation des features et de la cible
X = airbnb_train.copy()
X = X.drop(columns=['log_price'])
y = airbnb_train['log_price']

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Entraînement du modèle de régression linéaire
model = GradientBoostingRegressor()
model.fit(X_train, y_train)

# Prédiction du log_price sur l'ensemble de test
y_pred_train=model.predict(X_train)
y_pred_test = model.predict(X_test)

# Calcul de l'erreur quadratique moyenne
mse_test = mean_squared_error(y_test, y_pred_test)
print("Erreur quadratique moyenne en test :", mse_test)

mse_train = mean_squared_error(y_train, y_pred_train)
print("Erreur quadratique moyenne en train :", mse_train)

from sklearn.metrics import r2_score

print(f"Score en entrainenement : {r2_score(y_true=y_train, y_pred=y_pred_train)}")
print(f"Score en test : {r2_score(y_true=y_test, y_pred=y_pred_test)}")

## AdaBoostRegressor

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

# Séparation des features et de la cible
X = airbnb_train.copy()
X = X.drop(columns=['log_price'])
y = airbnb_train['log_price']

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Entraînement du modèle de régression linéaire
model = AdaBoostRegressor()
model.fit(X_train, y_train)

# Prédiction du log_price sur l'ensemble de test
y_pred_train=model.predict(X_train)
y_pred_test = model.predict(X_test)

# Calcul de l'erreur quadratique moyenne
mse_test = mean_squared_error(y_test, y_pred_test)
print("Erreur quadratique moyenne en test :", mse_test)

mse_train = mean_squared_error(y_train, y_pred_train)
print("Erreur quadratique moyenne en train :", mse_train)

from sklearn.metrics import r2_score

print(f"Score en entrainenement : {r2_score(y_true=y_train, y_pred=y_pred_train)}")
print(f"Score en test : {r2_score(y_true=y_test, y_pred=y_pred_test)}")

## Test de classe pour essayer tous les modèles

Il y a beaucoup de modèles à essayer avec de nombreux jeux de données différents (avec plus ou moins de variables) donc on va créer une classe qui prend en paramètre un dataset et un modèle pour tester plus rapidement les différents modèles avec les différents datasets

In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# import des models
from sklearn.linear_model import Ridge, LinearRegression, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor


def evaluate_model(model, dataset):
    # Séparation des features et de la cible
    X = dataset.drop(columns=['log_price'])
    y = dataset['log_price']

    # Division des données en ensembles d'entraînement et de test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Entraînement du modèle
    model.fit(X_train, y_train)

    # Prédictions
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    # Calcul de l'erreur quadratique moyenne pour l'ensemble de test et d'entrainement
    mse_test = mean_squared_error(y_test, y_pred_test)
    print(f"Erreur quadratique moyenne en test pour {model.__class__.__name__}: {mse_test}")
    mse_train = mean_squared_error(y_train, y_pred_train)
    print(f"Erreur quadratique moyenne en train pour {model.__class__.__name__}: {mse_train}")

    # Calcul du score R^2
    r2_train = r2_score(y_train, y_pred_train)
    print(f"Score R^2 en entraînement pour {model.__class__.__name__}: {r2_train}")
    r2_test = r2_score(y_test, y_pred_test)
    print(f"Score R^2 en test pour {model.__class__.__name__}: {r2_test}")
    print()



In [40]:
# Liste des modèles à évaluer
models = [
    LinearRegression(),
    Ridge(),
    Lasso(),
    ElasticNet(),
    SVR(),
    DecisionTreeRegressor(),
    RandomForestRegressor(),
    GradientBoostingRegressor(),
    AdaBoostRegressor(),
]



In [ ]:
# Évaluation des modèles pour le dataset avec toutes les variables
for model in models:
    evaluate_model(model, airbnb_train)

In [ ]:
# Évaluation des modèles pour le dataset avec toutes les variables, mais les lignes filtrées (les lignes avec trop de 0 ont été supprimées)
for model in models:
    evaluate_model(model, airbnb_filtre)

In [ ]:
# Évaluation des modèles pour le dataset avec les variables choisies d'après la matrice de corrélation
for model in models:
    evaluate_model(model, airbnb_matrice_corr)

In [ ]:
# Évaluation des modèles pour le dataset avec les variables choisies d'après les représentations graphiques
for model in models:
    evaluate_model(model, airbnb_graphique)


In [ ]:
# Évaluation des modèles pour le dataset avec les variables choisies par intuition
for model in models:
    evaluate_model(model, airbnb_utile)

# 3. Conclusion


On cherche le couple (modèle,dataset) tel que l'erreur quadratique soit le plsu près de 0 et le score r^2 soir le plus proche de 1. Il faut également que l'écart entre le score en entraînement et le score en test soit minime.
Ainsi, nous obtenons les meilleurs résultats avec le modèle DecisionTreeRegressor avec le dataset airbnb_train qui correspond à notre dataset le plus complet.

A noter que dans le cadre de notre projet, nous n'avions pas de consignes ou de contraintes concernant notre dataset. Dans la réalité, il est souvent nécéssaire d'éliminer des colonnes par souci d'efficacité, de stockage, de temps de traitement, etc. Mais il est normal que plus on garde de variable, plus les prédictions seront justes. C'est pourquoi, ici, c'est avec notre dataset le plus complet que nous obtenons les meilleurs scores. Dans un milieu plus réaliste, il aurait fallu choisir le dataset airbnb_matrice_corr, qui correspond aux variables choisies suite à l'étude de la matrice de corrélation. Mais les dimensions n'ont en effet rien à voir : 5 colonnes pour airbnb_matrice_corr contre 30 pour airbnb_train.

In [ ]:
#Récapitulatif modèle et dataset choisi
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor

# Séparation des features et de la cible
X = airbnb_train.copy()
X = X.drop(columns=['log_price'])
y = airbnb_train['log_price']

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Entraînement du modèle de régression linéaire
model = DecisionTreeRegressor()
model.fit(X_train, y_train)

# Prédiction du log_price sur l'ensemble de test
y_pred_train=model.predict(X_train)
y_pred_test = model.predict(X_test)

# Calcul de l'erreur quadratique moyenne
mse_test = mean_squared_error(y_test, y_pred_test)
print("Erreur quadratique moyenne en test :", mse_test)

mse_train = mean_squared_error(y_train, y_pred_train)
print("Erreur quadratique moyenne en train :", mse_train)

from sklearn.metrics import r2_score

print(f"Score en entrainenement : {r2_score(y_true=y_train, y_pred=y_pred_train)}")
print(f"Score en test : {r2_score(y_true=y_test, y_pred=y_pred_test)}")

In [ ]:
airbnb_test = pd.read_csv("airbnb_test.csv")

# J’applique le même traitement que mon fichier entraînement
final_X_test = features_transformer.transform(airbnb_test)

columns0 = ["property_type","room_type", "bathrooms", "bed_type", "cancellation_policy", "cleaning_fee","first_review","host_has_profile_pic",
           "host_identity_verified","host_response_rate","host_since","instant_bookable", "last_review", "latitude","longitude","number_of_reviews","review_scores_rating",
            "bedrooms","beds", "bathroom","bedroom","kitchen","heating","outdoor","tv/wifi","smoking","security","pets","essentials"]

feature_selector = FeatureSelection(columns0)
final_X_test = feature_selector.transform(final_X_test)

final_X_test.tail()

In [ ]:
y_final_prediction = model.predict(final_X_test)
print(y_final_prediction)

In [ ]:
#sauvegarde
# Créez un DataFrame avec les IDs et les prédictions
prediction_airbnb = pd.DataFrame({
    'id': airbnb_test.iloc[:, 0],
    'logpred': y_final_prediction
})

# Sauvegardez le DataFrame dans un fichier CSV
prediction_airbnb.to_csv("MaPredictionFinale.csv", index=False) # index=False pour éviter d’ajouter l’index interne à pandas

print("Les prédictions ont été sauvegardées dans 'MaPredictionFinale.csv'.")

In [ ]:
#Vérification conformité
def estConforme(monFichier_csv):
    votre_prediction = pd.read_csv(monFichier_csv)

    fichier_exemple = pd.read_csv("prediction_example.csv")

    assert votre_prediction.columns[1] == fichier_exemple.columns[1], f"Attention, votre colonne de prédiction doit s'appeler {fichier_exemple.columns[1]}, elle s'appelle '{votre_prediction.columns[1]}'"
    assert len(votre_prediction) == len(fichier_exemple), f"Attention, vous devriez avoir {len(fichier_exemple)} prédiction dans votre fichier, il en contient '{len(votre_prediction)}'"

    assert np.all(votre_prediction.iloc[:,0] == fichier_exemple.iloc[:, 0])

    print("Fichier conforme!")

estConforme("MaPredictionFinale.csv")